In [29]:
from azureml.core import Workspace

In [30]:
subscription_id = "d74ca76c-0758-4302-8894-12939776ca14" # The ID of the Azure Subscription
resource_group = "fastai-practise" # Name of a logical resource group
workspace_name = "Azure_projects" # The name of the workspace to look for or to create
workspace_region = 'northcentralus' # Location of the workspace
experiment_name = 'cars-classifier'
score_script = 'score_and_track.py'

In [33]:
ws = Workspace.create(name=workspace_name,
               subscription_id=subscription_id,
               resource_group=resource_group,
               create_resource_group=True,
               location=workspace_region,
               )

Deploying KeyVault with name azureprokeyvault49c17beb.
Deploying StorageAccount with name azureprostorage7558cdf32.
Deploying AppInsights with name azureproinsights49a1f224.
Deployed AppInsights with name azureproinsights49a1f224. Took 1.37 seconds.
Deployed KeyVault with name azureprokeyvault49c17beb. Took 17.16 seconds.
Deploying Workspace with name Azure_projects.
Deployed StorageAccount with name azureprostorage7558cdf32. Took 20.83 seconds.
Deployed Workspace with name Azure_projects. Took 20.88 seconds.


In [34]:
# ws = Workspace.get(name="Azure_projects", subscription_id='d74ca76c-0758-4302-8894-12939776ca14', resource_group='fastai-practise')

In [35]:
from azureml.core.model import Model

model = Model.register(model_path="./cars.pkl",
                          model_name="cars-classifier",
                          tags={'data': 'cars',  'method':'transfer learning', 'framework':'pytorch'},
                          description='Image classficiation service BMW vs Jagaur vs Mclauren',
                          workspace=ws)

Registering model cars-classifier


In [37]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment
from azureml.core.model import InferenceConfig

myenv = CondaDependencies.create(pip_packages=['azureml-defaults', 'fastai==1.0.60','pillow==5.4.1',
                                               'torch', 'torchvision>=0.5.0'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

# myenv = CondaDependencies()
# myenv.add_conda_package("fastai")
# myenv.add_conda_package("pytorch")
# myenv.add_conda_package("torchvision")
# myenv.add_pip_package("azureml-defaults")
# myenv.add_channel("pytorch")
# myenv.add_channel("fastai")
# conda_file_name='myenv.yml'

# with open(conda_file_name,"w") as f:
#     f.write(myenv.serialize_to_string())
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score_and_track.py",
                                   environment=myenv)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults
  - fastai==1.0.60
  - pillow==5.4.1
  - torch
  - torchvision>=0.5.0
channels:
- conda-forge



In [40]:
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException


service_name = 'cars'

# Remove any existing service under the same name.
try:
   Webservice(ws, service_name).delete()
except WebserviceException:
   pass
    
aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=4, 
                                               tags={'data': 'cars',  'method':'transfer learning', 'framework':'pytorch'},
                                               description='Image classficiation service BMW vs Jagaur vs Mclauren')

In [41]:
service = Model.deploy(workspace=ws, 
                           name=service_name, 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

Running.......................................

ERROR - Received bad response from Resource Provider:
Response Code: 500
Headers: {'Date': 'Wed, 11 Mar 2020 16:37:40 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"InternalServerError","statusCode":500,"message":"An internal server error occurred. Please try again. If the problem persists, contact support"}'

ERROR - Received bad response from Resource Provider:
Response Code: 500
Headers: {'Date': 'Wed, 11 Mar 2020 16:37:40 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"InternalServerError","statusCode":500,"message":"An internal server error o

WebserviceException: WebserviceException:
	Message: Received bad response from Resource Provider:
Response Code: 500
Headers: {'Date': 'Wed, 11 Mar 2020 16:37:40 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"InternalServerError","statusCode":500,"message":"An internal server error occurred. Please try again. If the problem persists, contact support"}'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from Resource Provider:\nResponse Code: 500\nHeaders: {'Date': 'Wed, 11 Mar 2020 16:37:40 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'Strict-Transport-Security': 'max-age=15724800; includeSubDom

In [42]:
service.get_logs()

ERROR - Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Wed, 11 Mar 2020 16:39:41 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-client-request-id': '2e96eae0112b4c63bd52c625ded03be7', 'x-ms-client-session-id': '', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found","details":[{"code":"ResourceNotFound","message":"The Resource \'Microsoft.ContainerInstance/containerGroups/cars\' under resource group \'fastai-practise\' was not found."}]}'



WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 404
Headers: {'Date': 'Wed, 11 Mar 2020 16:39:41 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-client-request-id': '2e96eae0112b4c63bd52c625ded03be7', 'x-ms-client-session-id': '', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'Content-Encoding': 'gzip'}
Content: b'{"code":"NotFound","statusCode":404,"message":"The specified resource was not found","details":[{"code":"ResourceNotFound","message":"The Resource \'Microsoft.ContainerInstance/containerGroups/cars\' under resource group \'fastai-practise\' was not found."}]}'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response 

In [43]:
print(service.scoring_uri)

None
